## <p style="text-align:center" color="red"><span style="color:red">Python Web Scraping with Selenium: Quran Karim french version</span></p>


<table align="center">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/labrijisaad/Python-Web-Scraping-Quran-Karim-with-Selenium"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

- 🎯 In this notebook, I tried to write a script able to scrape with Selenium the Koran Karim in French and to save it as CSV file.
- 🧹 This notebook also cleans the dataset before saving it to a CSV file (the cleaning process takes care of unnecessary numbers, punctuation, excessive white spaces...

-  📍  This is the website we used for our scrapping process → **[le-coran](https://www.le-coran.com/coran-francais-sourate-15-0.html)**
- 📫 Feel free to contact me if anything is wrong or if anything needs to be changed 😎!  **labrijisaad@gmail.com**

### Installing the necessary dependencies

In [1]:
%%capture
# install chromedriver and selenium
!pip install chromedriver-py==101.0.4951.41
!pip install selenium
!pip install matplotlib

### Importing the necessary libraries

In [2]:
# importing necessary packages
from __future__ import unicode_literals
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from chromedriver_py import binary_path
import re
import time
from selenium import webdriver
from tqdm import tqdm
import os
import matplotlib.pyplot as plt
%matplotlib inline
# Don't Show Warning Messages

C:\Users\SAAD\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\SAAD\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\SAAD\anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


### Setting up the selenium environment

In [3]:
# creating a function that creates a chrome options object with given parameters
def add_driver_options(options):
    """
    This function sets driver options ( given as parameter)
    """
    chrome_options = Options()
    for opt in options:
        chrome_options.add_argument(opt)
    return chrome_options

# create function that initialize the driver and return it

def initialize_driver():
    """
    This function create an instace of the web driver with the options below:
    """
    driver_config = {
        "executable_path": binary_path,
        "options": [
            "--headless",
            "--no-sandbox",
            "--start-fullscreen",
            "--allow-insecure-localhost",
            "--disable-dev-shm-usage",
        ],
    }
    options = add_driver_options(driver_config["options"])
    driver = webdriver.Chrome(
        executable_path=driver_config["executable_path"], options=options
    )
    return driver

In [4]:
%%capture
driver = initialize_driver() # create a driver with initialize_driver method

In [5]:
driver.get("https://www.le-coran.com/coran-francais-sourate-15-0.html")
elements = driver.find_elements(by=By.XPATH, value="/html[1]/body[1]/div[3]/div[1]/div[3]/div[4]/div[1]/div[1]/p")[1:]

### Performing the scrapping

In [6]:
sourat_dict = {}
sourat_verses_list = []

for i in tqdm(range(1, 115)):
    driver.get("https://www.le-coran.com/coran-francais-sourate-"+str(i)+"-0.html")
    if i != 18:
        scrapped_verses = driver.find_elements(by=By.XPATH, value="/html[1]/body[1]/div[3]/div[1]/div[3]/div[4]/div[1]/div[1]/p")[1:]
        sourat_verses_list = [verse.text for verse in scrapped_verses]
        sourat_name = driver.find_element(by=By.XPATH, value="/html[1]/body[1]/div[3]/div[1]/div[3]/div[1]/h1[1]").text
        sourat_dict[sourat_name] = sourat_verses_list
    else:
        scrapped_verses = driver.find_elements(by=By.XPATH, value="/html[1]/body[1]/div[3]/div[1]/div[3]/div[5]/div[1]/div[1]/p")[1:]
        sourat_verses_list = [verse.text for verse in scrapped_verses]
        sourat_name = driver.find_element(by=By.XPATH, value="/html[1]/body[1]/div[3]/div[1]/div[3]/div[1]/h1[1]").text
        sourat_dict[sourat_name] = sourat_verses_list
        

100%|████████████████████████████████████████████████████████████████████████████████| 114/114 [03:44<00:00,  1.97s/it]


In [7]:
for key, value in sourat_dict.items():
    print(key, value[:2], "\n\n")

Sourate AL-FATIHA / PROLOGUE en français | Sourate 1 ["1 Au nom d'Allah, le Tout Miséricordieux, le Très Miséricordieux.", "2 Louange à Allah, Seigneur de l'univers."] 


Sourate AL-BAQARA / LA VACHE en français | Sourate 2 ['1 Alif, Lam, Mim.', "2 C'est le Livre au sujet duquel il n'y a aucun doute, c'est un guide pour les pieux,"] 


Sourate AL-IMRAN / LA FAMILLE D'IMRAN en français | Sourate 3 ['1 Alif, Lam, Mim.', '2 Allah ! Pas de divinité à part Lui, le Vivant, Celui qui subsiste par Lui-même: « al-Qayyum ».'] 


Sourate AN-NISA' / LES FEMMES en français | Sourate 4 ["1 Ô hommes ! Craignez votre Seigneur qui vous a créés d'un seul être, et a créé de celui-ci son épouse, et qui de ces deux là a fait répandre (sur la terre) beaucoup d'hommes et de femmes. Craignez Allah au nom duquel vous vous implorez les uns les autres, et craignez de rompre les liens du sang. Certes Allah vous observe parfaitement.", "2 Et donnez aux orphelins leurs biens; n'y substituez pas le mauvais au bon. N

Everything we scrapped is stored in the dictionary: **`sourat_dict`**
### Turning the dictionary into pandas dataframe

In [8]:
import pandas as pd
df = pd.DataFrame()

for sourat, verses in sourat_dict.items():
    print(sourat)
    for verse in verses:
        index_verse = ''.join([n for n in verse if n.isdigit()])
        index_sourat = ''.join([n for n in sourat if n.isdigit()])
        row = {'index sourat':index_sourat, 'index verse':index_verse, 'name sourat in french':sourat, 'verse in french':verse}        
        df = df.append(row, ignore_index=True)

Sourate AL-FATIHA / PROLOGUE en français | Sourate 1
Sourate AL-BAQARA / LA VACHE en français | Sourate 2
Sourate AL-IMRAN / LA FAMILLE D'IMRAN en français | Sourate 3
Sourate AN-NISA' / LES FEMMES en français | Sourate 4
Sourate AL-MAIDAH / LA TABLE SERVIE en français | Sourate 5
Sourate AL-ANAM / LES BESTIAUX en français | Sourate 6
Sourate AL-ARAF / LES MURAILLES en français | Sourate 7
Sourate AL-ANFAL / LE BUTIN en français | Sourate 8
Sourate AT-TAWBAH / LE REPENTIR en français | Sourate 9
Sourate YUNUS / JONAS en français | Sourate 10
Sourate HUD / HUD en français | Sourate 11
Sourate YUSUF / JOSEPH en français | Sourate 12
Sourate AR-RAAD / LE TONNERRE en français | Sourate 13
Sourate IBRAHIM / ABRAHAM en français | Sourate 14
Sourate AL-HIJR / AL-HIJR en français | Sourate 15
Sourate AN-NAHL / LES ABEILLES en français | Sourate 16
Sourate AL-ISRA / LE VOYAGE NOCTURNE en français | Sourate 17
Sourate AL-KAHF / LA CAVERNE en français | Sourate 18
Sourate MARYAM / MARIE en frança

In [9]:
df

,index sourat,index verse,name sourat in french,verse in french
0,1,1,Sourate AL-FATIHA / PROLOGUE en français | Sou...,"1 Au nom d'Allah, le Tout Miséricordieux, le T..."
1,1,2,Sourate AL-FATIHA / PROLOGUE en français | Sou...,"2 Louange à Allah, Seigneur de l'univers."
2,1,3,Sourate AL-FATIHA / PROLOGUE en français | Sou...,"3 Le Tout Miséricordieux, le Très Miséricordieux,"
3,1,4,Sourate AL-FATIHA / PROLOGUE en français | Sou...,4 Maître du Jour de la rétribution.
4,1,5,Sourate AL-FATIHA / PROLOGUE en français | Sou...,"5 C'est Toi [Seul] que nous adorons, et c'est ..."
...,...,...,...,...
6231,114,2,Sourate AN-NAS / LES HOMMES en français | Sour...,"2 Le Souverain des hommes,"
6232,114,3,Sourate AN-NAS / LES HOMMES en français | Sour...,"3 Dieu des hommes,"
6233,114,4,Sourate AN-NAS / LES HOMMES en français | Sour...,"4 contre le mal du mauvais conseiller, furtif,"
6234,114,5,Sourate AN-NAS / LES HOMMES en français | Sour...,5 qui souffle le mal dans les poitrines des ho...


In [10]:
sourat_lenSourat = [(1, 7), (2, 286), (3, 200), (4, 176), (5, 120), (6, 165), (7, 206), (8, 75), (9, 129), (10, 109), (11, 123), (12, 111), (13, 43), (14, 52), (15, 99), (16, 128), (17, 111), (18, 110), (19, 98), (20, 135), (21, 112), (22, 78), (23, 118), (24, 64), (25, 77), (26, 227), (27, 93), (28, 88), (29, 69), (30, 60), (31, 34), (32, 30), (33, 73), (34, 54), (35, 45), (36, 83), (37, 182), (38, 88), (39, 75), (40, 85), (41, 54), (42, 53), (43, 89), (44, 59), (45, 37), (46, 35), (47, 38), (48, 29), (49, 18), (50, 45), (51, 60), (52, 49), (53, 62), (54, 55), (55, 78), (56, 96), (57, 29), (58, 22), (59, 24), (60, 13), (61, 14), (62, 11), (63, 11), (64, 18), (65, 12), (66, 12), (67, 30), (68, 52), (69, 52), (70, 44), (71, 28), (72, 28), (73, 20), (74, 56), (75, 40), (76, 31), (77, 50), (78, 40), (79, 46), (80, 42), (81, 29), (82, 19), (83, 36), (84, 25), (85, 22), (86, 17), (87, 19), (88, 26), (89, 30), (90, 20), (91, 15), (92, 21), (93, 11), (94, 8), (95, 8), (96, 19), (97, 5), (98, 8), (99, 8), (100, 11), (101, 11), (102, 8), (103, 3), (104, 9), (105, 5), (106, 4), (107, 7), (108, 3), (109, 6), (110, 3), (111, 5), (112, 4), (113, 5), (114, 6)]  

The **`sourat_lenSourat`** is a list that contains tuples composed of the index of the surat and the verse number of this surat, this list will be used to check if there is a problem with Scrapping and to avoid having surats with erroneous verses.

In [11]:
for tpl in sourat_lenSourat:    
    if tpl[1] != len(df[df["index sourat"]==str(tpl[0])]): 
        print(False)
        print(tpl)

### Cleaning the dataset

In [12]:
def remove_digits(text):
    res = ''.join([i for i in text if not i.isdigit()])
    return res

df['verse in french'] = df['verse in french'].apply(lambda text: remove_digits(text))

In [13]:
df

,index sourat,index verse,name sourat in french,verse in french
0,1,1,Sourate AL-FATIHA / PROLOGUE en français | Sou...,"Au nom d'Allah, le Tout Miséricordieux, le Tr..."
1,1,2,Sourate AL-FATIHA / PROLOGUE en français | Sou...,"Louange à Allah, Seigneur de l'univers."
2,1,3,Sourate AL-FATIHA / PROLOGUE en français | Sou...,"Le Tout Miséricordieux, le Très Miséricordieux,"
3,1,4,Sourate AL-FATIHA / PROLOGUE en français | Sou...,Maître du Jour de la rétribution.
4,1,5,Sourate AL-FATIHA / PROLOGUE en français | Sou...,"C'est Toi [Seul] que nous adorons, et c'est T..."
...,...,...,...,...
6231,114,2,Sourate AN-NAS / LES HOMMES en français | Sour...,"Le Souverain des hommes,"
6232,114,3,Sourate AN-NAS / LES HOMMES en français | Sour...,"Dieu des hommes,"
6233,114,4,Sourate AN-NAS / LES HOMMES en français | Sour...,"contre le mal du mauvais conseiller, furtif,"
6234,114,5,Sourate AN-NAS / LES HOMMES en français | Sour...,qui souffle le mal dans les poitrines des hom...


In [14]:
sourat_names = df['name sourat in french'].str.split("/", expand = True)
df['name sourat in french'] = sourat_names[0]
df['name sourat in french'] = df['name sourat in french'].apply(lambda text: text[8:])

df

,index sourat,index verse,name sourat in french,verse in french
0,1,1,AL-FATIHA,"Au nom d'Allah, le Tout Miséricordieux, le Tr..."
1,1,2,AL-FATIHA,"Louange à Allah, Seigneur de l'univers."
2,1,3,AL-FATIHA,"Le Tout Miséricordieux, le Très Miséricordieux,"
3,1,4,AL-FATIHA,Maître du Jour de la rétribution.
4,1,5,AL-FATIHA,"C'est Toi [Seul] que nous adorons, et c'est T..."
...,...,...,...,...
6231,114,2,AN-NAS,"Le Souverain des hommes,"
6232,114,3,AN-NAS,"Dieu des hommes,"
6233,114,4,AN-NAS,"contre le mal du mauvais conseiller, furtif,"
6234,114,5,AN-NAS,qui souffle le mal dans les poitrines des hom...


In [15]:
def remove_ponctuation(text):
    for ele in text:
        if ele in '''!()-[]{}»«;:'"\,<>./?@#$%^&*_~''':
            text = text.replace(ele, " ")
    return text

df['verse in french'] = df['verse in french'].apply(lambda x: remove_ponctuation(x))
df['name sourat in french'] = df['name sourat in french'].apply(lambda x: remove_ponctuation(x))

In [16]:
# lower()
df['verse in french'] = df['verse in french'].str.lower()
df['name sourat in french'] = df['name sourat in french'].str.lower()

In [17]:
# remove excess whitespace 
df['verse in french'] = df['verse in french'].apply(lambda text: re.sub("\s\s+", " ", text))
df['name sourat in french'] = df['name sourat in french'].apply(lambda text: re.sub("\s\s+", " ", text))
df

,index sourat,index verse,name sourat in french,verse in french
0,1,1,al fatiha,au nom d allah le tout miséricordieux le très...
1,1,2,al fatiha,louange à allah seigneur de l univers
2,1,3,al fatiha,le tout miséricordieux le très miséricordieux
3,1,4,al fatiha,maître du jour de la rétribution
4,1,5,al fatiha,c est toi seul que nous adorons et c est toi ...
...,...,...,...,...
6231,114,2,an nas,le souverain des hommes
6232,114,3,an nas,dieu des hommes
6233,114,4,an nas,contre le mal du mauvais conseiller furtif
6234,114,5,an nas,qui souffle le mal dans les poitrines des hom...


### Saving the dataframe

In [18]:
df.to_csv("../Coran data-set/Coran french processed.csv", encoding="utf-8-sig", index=False)

### Reading the dataset again

In [19]:
df = pd.read_csv("../Coran data-set/Coran french processed.csv", encoding='utf-8')
df

,index sourat,index verse,name sourat in french,verse in french
0,1,1,al fatiha,au nom d allah le tout miséricordieux le très...
1,1,2,al fatiha,louange à allah seigneur de l univers
2,1,3,al fatiha,le tout miséricordieux le très miséricordieux
3,1,4,al fatiha,maître du jour de la rétribution
4,1,5,al fatiha,c est toi seul que nous adorons et c est toi ...
...,...,...,...,...
6231,114,2,an nas,le souverain des hommes
6232,114,3,an nas,dieu des hommes
6233,114,4,an nas,contre le mal du mauvais conseiller furtif
6234,114,5,an nas,qui souffle le mal dans les poitrines des hom...


> - 🙌 Notebook made by [@labriji_saad](https://github.com/labrijisaad)
> - 🔗 Linledin [@labriji_saad](https://www.linkedin.com/in/labrijisaad/)